In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
#import dataframe as df


movie = pd.read_csv('movie_data_for_assignment.csv')

In [2]:
movie.head


<bound method NDFrame.head of                                      movie_title             color  \
0                                 Stuart Little              Color   
1                                         Crash              Color   
2              Ghost Rider: Spirit of Vengeance              Color   
3                                    Gettysburg              Color   
4                            Planet of the Apes              Color   
5                         Daredevil                            NaN   
6                      The Pursuit of Happyness              Color   
7                           Kiss Kiss Bang Bang              Color   
8                              Music and Lyrics              Color   
9                               Horrible Bosses              Color   
10                                    Blindness              Color   
11                                    Cleopatra              Color   
12                                  Out of Time             

In [3]:
str_list = [] # empty list to contain columns with strings (words)
for colname, colvalue in movie.iteritems():
    if type(colvalue[1]) == str:
         str_list.append(colname)
# Get to the numeric columns by inversion            
num_list = movie.columns.difference(str_list)         

In [4]:
movie_num = movie[num_list]
#del movie # Get rid of movie df as we won't need it now
movie_num.head()
numpyMatrix = movie_num.as_matrix()
print(numpyMatrix.shape)
y_std = numpyMatrix[:, [10]]
nump1 = np.delete(numpyMatrix, [10], axis=1)
movie_num.shape;
print(nump1.shape)
movie_num1=movie_num.drop('imdb_score', axis=1, inplace=True) 


(4300, 16)
(4300, 15)


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
movie_num = movie_num.fillna(value=0, axis=1)

In [6]:
movie_num = movie_num.fillna(value=0, axis=1)

In [7]:
X = movie_num.values
# Data Normalization
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)
#y_std=X_std[:, [15]]
print(y_std)

[[ 5.9]
 [ 7.9]
 [ 4.3]
 ..., 
 [ 7.2]
 [ 3.9]
 [ 7.1]]


In [8]:
# Calculating Eigenvectors and eigenvalues of Cov matirx
mean_vec = np.mean(X_std, axis=0)
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)

In [9]:
# Create a list of (eigenvalue, eigenvector) tuples
eig_pairs = [ (np.abs(eig_vals[i]),eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort from high to low
eig_pairs.sort(key = lambda x: x[0], reverse= True)

# Calculation of Explained Variance from the eigenvalues
tot = sum(eig_vals)
var_exp = [(i/tot)*100 for i in sorted(eig_vals, reverse=True)] # Individual explained variance
cum_var_exp = np.cumsum(var_exp) # Cumulative explained variance

In [10]:
pca = PCA(n_components=2)
x_9d = pca.fit_transform(X_std)

In [11]:
print(x_9d.shape)

(4300, 2)


In [12]:
ys = y_std
xs = x_9d

In [13]:
x_train = xs[range(3300),:]
y_train = ys[range(3300),:]
#x_valid = xs[range(2300,3300),:]
#y_valid = ys[range(2300,3300),:]
x_test = xs[range(3300,4300),:]
y_test = ys[range(3300,4300),:]


n_observations = 2300
fig, ax = plt.subplots(1, 1)
x_train = xs[range(2300),:]
y_train = ys[range(2300),:]
x_valid = xs[range(2300,3300),:]
y_valid = ys[range(2300,3300),:]
x_test = xs[range(3300,3400),:]
y_test = ys[range(3300,3400),:]
print(y_test.shape)

In [ ]:
# %% Let's create some toy data
plt.ion()
n_observations = 2300
fig, ax = plt.subplots(1, 1)
x_train = xs[range(3300),:]
y_train = ys[range(3300),:]
#x_valid = xs[range(2300,3300),:]
#y_valid = ys[range(2300,3300),:]
x_test = xs[range(3300,4300),:]
y_test = ys[range(3300,4300),:]
print(y_test.shape)
#xs = np.linspace(-3, 3, n_observations)
#ys = np.sin(xs) + np.random.uniform(-0.5, 0.5, n_observations)
#ax.scatter(xs, ys)
#fig.show()
#plt.draw()

# %% tf.placeholders for the input and output of the network. Placeholders are
# variables which we need to fill in when we are ready to compute the graph.
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# %% We will try to optimize min_(W,b) ||(X*w + b) - y||^2
# The `Variable()` constructor requires an initial value for the variable,
# which can be a `Tensor` of any type and shape. The initial value defines the
# type and shape of the variable. After construction, the type and shape of
# the variable are fixed. The value can be changed using one of the assign
# methods.
W = tf.Variable(tf.random_normal([1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')
Y_pred = tf.add(tf.mul(X, W), b)


# %% Loss function will measure the distance between our observations
# and predictions and average over them.
cost = tf.reduce_sum(tf.pow(Y_pred - Y, 2)) / (n_observations - 1)

# %% if we wanted to add regularization, we could add other terms to the cost,
# e.g. ridge regression has a parameter controlling the amount of shrinkage
# over the norm of activations. the larger the shrinkage, the more robust
# to collinearity.
# cost = tf.add(cost, tf.mul(1e-6, tf.global_norm([W])))

# %% Use gradient descent to optimize W,b
# Performs a single step in the negative gradient
correct_prediction = tf.equal(Y_pred,Y);
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# %% We create a session to use the graph
n_epochs = 100
with tf.Session() as sess:
    # Here we tell tensorflow that we want to initialize all
    # the variables in the graph so we can use them
    sess.run(tf.initialize_all_variables())

    # Fit all training data
    prev_training_cost = 0.0
    for epoch_i in range(n_epochs):
        for (x, y) in zip(x_train, y_train):
            sess.run(optimizer, feed_dict={X: x, Y: y})

        training_cost = sess.run(
            cost, feed_dict={X: x_train, Y: y_train})
        print(training_cost)
        if np.abs(prev_training_cost - training_cost) < 0.000001:
         break
        prev_training_cost = training_cost
        
  #  print(sess.run(accuracy,
   #     feed_dict={
    #    X: x_valid,
     #   Y: y_valid
      #  }))

# %% Print final test accuracy:
  #  print(sess.run(accuracy,
   #     feed_dict={
    #    X: x_test,
     #   Y: y_test
      # }))
        testing_cost = sess.run(
        tf.reduce_sum(tf.pow(Y_pred - Y, 2)) / (2 * x_test.shape[0]),
        feed_dict={X:x_test, Y:y_test})  # same function as cost above
    print("Testing cost=", testing_cost)
    print("Absolute mean square loss difference:", abs(
        training_cost - testing_cost)) 
        
    plt.plot(x_test, y_test, 'bo', label='Testing data')
    plt.plot(x_train, sess.run(W) * x_train + sess.run(b), label='Fitted line')
    plt.legend()
    plt.show()
        
        
       # if epoch_i % 20 == 0:
        #    ax.plot(xs, Y_pred.eval(
        #        feed_dict={X: xs}, session=sess),
         #           'k', alpha=epoch_i / n_epochs)
         #   fig.show()
         #   plt.draw()

        # Allow the training to quit if we've reached a minimum
       # if np.abs(prev_training_cost - training_cost) < 0.000001:
       #     break
       # prev_training_cost = training_cost
#fig.show()
#plt.waitforbuttonpress()

In [ ]:
def create_samples(n_clusters, n_samples_per_cluster, n_features, embiggen_factor, seed):
    np.random.seed(seed)
    slices = []
    centroids = []
    # Create samples for each cluster
    for i in range(n_clusters):
        samples = tf.random_normal((n_samples_per_cluster, n_features),
                               mean=0.0, stddev=5.0, dtype=tf.float32, seed=seed, name="cluster_{}".format(i))
        current_centroid = (np.random.random((1, n_features)) * embiggen_factor) - (embiggen_factor/2)
        centroids.append(current_centroid)
        samples += current_centroid
        slices.append(samples)
    # Create a big "samples" dataset
    samples = tf.concat(0, slices, name='samples')
    centroids = tf.concat(0, centroids, name='centroids')
   # xs = tf.cast(xs, dtype="float32", name=None)
    return centroids, samples

In [ ]:


n_features = 2
n_clusters = 3
n_samples_per_cluster = 500
seed = 700
embiggen_factor = 70
np.random.seed(seed)

centroids, samples = create_samples(n_clusters, n_samples_per_cluster, n_features, embiggen_factor, seed)
samples = tf.convert_to_tensor(samples)
model = tf.initialize_all_variables()
with tf.Session() as session:
    sample_values = session.run(samples)
    centroid_values = session.run(centroids)

In [ ]:
def plot_clusters(all_samples, centroids, n_samples_per_cluster):
    import matplotlib.pyplot as plt
    # Plot out the different clusters
    # Choose a different colour for each cluster
    colour = plt.cm.rainbow(np.linspace(0,1,len(centroids)))
    for i, centroid in enumerate(centroids):
        # Grab just the samples fpr the given cluster and plot them out with a new colour
        samples = all_samples[i*n_samples_per_cluster:(i+1)*n_samples_per_cluster]
        plt.scatter(samples[:,0], samples[:,1], c=colour[i])
        # Also plot centroid
        plt.plot(centroid[0], centroid[1], markersize=35, marker="x", color='k', mew=10)
        plt.plot(centroid[0], centroid[1], markersize=30, marker="x", color='m', mew=5)
    plt.show()

In [ ]:
def choose_random_centroids(samples, n_clusters):
    # Step 0: Initialisation: Select `n_clusters` number of random points
    n_samples = tf.shape(samples)[0]
    random_indices = tf.random_shuffle(tf.range(0, n_samples))
    begin = [0,]
    size = [n_clusters,]
    size[0] = n_clusters
    centroid_indices = tf.slice(random_indices, begin, size)
    initial_centroids = tf.gather(samples, centroid_indices)
    return initial_centroids
    

In [ ]:
n_features = 2
n_clusters = 3
n_samples_per_cluster = 500
seed = 700
embiggen_factor = 70

centroids, samples = create_samples(n_clusters, n_samples_per_cluster, n_features, embiggen_factor, seed)
initial_centroids = choose_random_centroids(samples, n_clusters)
xs = tf.convert_to_tensor(xs)
model = tf.initialize_all_variables()
with tf.Session() as session:
    sample_values = session.run(xs)
    updated_centroid_value = session.run(initial_centroids)

#plot_clusters(sample_values, updated_centroid_value, n_samples_per_cluster)

In [ ]:
def assign_to_nearest(samples, centroids):
    # Finds the nearest centroid for each sample

    # START from http://esciencegroup.com/2016/01/05/an-encounter-with-googles-tensorflow/
    expanded_vectors = tf.expand_dims(samples, 0)
    expanded_centroids = tf.expand_dims(centroids, 1)
    distances = tf.reduce_sum( tf.square(
               tf.sub(expanded_vectors, expanded_centroids)), 2)
    mins = tf.argmin(distances, 0)
    # END from http://e}sciencegroup.com/2016/01/05/an-encounter-with-googles-tensorflow/
    nearest_indices = mins
    return nearest_indices

In [ ]:
def update_centroids(samples, nearest_indices, n_clusters):
    # Updates the centroid to be the mean of all samples associated with it.
    nearest_indices = tf.to_int32(nearest_indices)
    partitions = tf.dynamic_partition(samples, nearest_indices, n_clusters)
    new_centroids = tf.concat(0, [tf.expand_dims(tf.reduce_mean(partition, 0), 0) for partition in partitions])
    return new_centroids

In [ ]:

n_features = 2
n_clusters = 3
n_samples_per_cluster = 500
seed = 700
embiggen_factor = 70

xs = tf.cast(xs, dtype="float32", name=None)
#data_centroids, samples = create_samples(n_clusters, n_samples_per_cluster, n_features, embiggen_factor, seed)
initial_centroids = choose_random_centroids(xs, n_clusters)
nearest_indices = assign_to_nearest(xs, initial_centroids)
updated_centroids = update_centroids(xs, nearest_indices, n_clusters)
for centroids_num in range(32):
    model = tf.initialize_all_variables()
    with tf.Session() as session:
        sample_values = session.run(xs)
        updated_centroid_value = session.run(updated_centroids)
        print(updated_centroid_value)

        plot_clusters(sample_values, updated_centroid_value, n_samples_per_cluster)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
print(updated_centroid_value.shape)
print(type(nearest_indices))

In [ ]:
ax.scatter(xs[:,[0]],xs[:,[1]],ys)
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

In [ ]:
from sklearn.cluster import KMeans 
kmeans = KMeans(n_clusters=3)
# Compute cluster centers and predict cluster indices
X_clustered = kmeans.fit_predict(xs)
print(X_clustered.shape)
# Define our own color map
LABEL_COLOR_MAP = {0 : 'r',1 : 'g',2 : 'b'}
label_color = [LABEL_COLOR_MAP[l] for l in X_clustered]

# Plot the scatter digram
plt.figure(figsize = (7,7))
plt.scatter(xs[:,0],xs[:,1], c= label_color, alpha=0.5) 
plt.show()


In [ ]:
X = tf.placeholder(tf.float32,name="X",shape=[None,2]);
W = tf.Variable(tf.random_normal([2,5],dtype=tf.float32,stddev = 0.1,name ="W"))
h = tf.matmul(X,W);
b = tf.Variable(tf.constant([0,1],dtype=tf.float32,shape=[5],name = "b"));
h = tf.nn.bias_add(h,b);
h = tf.nn.relu(h);
from libs import utils
def linear(x, n_output, name=None, activation=None, reuse=None):
    """Fully connected layer

    Parameters
    ----------
    x : tf.Tensor
        Input tensor to connect
    n_output : int
        Number of output neurons
    name : None, optional
        Scope to apply

    Returns
    -------
    op : tf.Tensor
        Output of fully connected layer.
    """
    if len(x.get_shape()) != 2:
        x = flatten(x, reuse=reuse)

    n_input = x.get_shape().as_list()[1]

    with tf.variable_scope(name or "fc", reuse=reuse):
        W = tf.get_variable(
            name='W',
            shape=[n_input, n_output],
            dtype=tf.float32,
            initializer=tf.contrib.layers.xavier_initializer())

        b = tf.get_variable(
            name='b',
            shape=[n_output],
            dtype=tf.float32,
            initializer=tf.constant_initializer(0.0))

        h = tf.nn.bias_add(
            name='h',
            value=tf.matmul(x, W),
            bias=b)

        if activation:
            h = activation(h)

        return h, W

In [ ]:
tf.reset_default_graph()

# Create a placeholder of None x 2 dimensions and dtype tf.float32
# This will be the input to the network which takes the row/col
X = tf.placeholder(tf.float32,name="X",shape=[None,2]);# Create the placeholder, Y, with 3 output dimensions instead of 2.
# This will be the output of the network, the R, G, B values.
Y = tf.placeholder(tf.float32,name="Y",shape=[None,10]);
labels = tf.placeholder(tf.float32,name="labels",shape=[None,10]);

In [ ]:
# We'll create 6 hidden layers.  Let's create a variable
# to say how many neurons we want for each of the layers
# (try 20 to begin with, then explore other values)
n_neurons = 20;

# Create the first linear + nonlinear layer which will
# take the 2 input neurons and fully connects it to 20 neurons.
# Use the `utils.linear` function to do this just like before,
# but also remember to give names for each layer, such as
# "1", "2", ... "5", or "layer1", "layer2", ... "layer6".
h1, W1 = linear(
    x=X, n_output=20, name='linear1', activation=tf.nn.relu)


# Create another one:
h2, W2 = linear(
    x=h1, n_output=20, name='linear2', activation=tf.nn.relu)

# and four more (or replace all of this with a loop if you can!):
h3, W3 =  linear(
    x=h2, n_output=20, name='linear3', activation=tf.nn.relu)
h4, W4 =  linear(
    x=h3, n_output=20, name='linear4', activation=tf.nn.relu)
h5, W5 =  linear(
    x=h4, n_output=20, name='linear5', activation=tf.nn.relu)
h6, W6 =  linear(
    x=h5, n_output=20, name='linear6', activation=tf.nn.relu)

# Now, make one last layer to make sure your network has 3 outputs:
Y_pred, W7 = linear(h6, 10, activation=None, name='pred')

In [ ]:
import math
print(ys[0])
len = ys.shape[0];
for r in range(len):
    ys[r]= math.ceil(ys[r])
    inputs, num_labels = ys[r], 10
    indexedInputs = [[i, inputs[i]] for i in range(inputs.shape[0])]
    indexedInputs =tf.cast(indexedInputs, tf.int32)
    with tf.Session() as session:
        print(indexedInputs.eval())
        ys[r]=tf.sparse_to_dense(indexedInputs[0],indexedInputs[1] ,[1,num_labels], 1) 

In [ ]:
import math
for r in range(4300):
    ys[r]= math.ceil(ys[r])
print(ys.shape())

In [ ]:
ys = np.asarray(ys)
num_labels = 10
sparse_labels = tf.reshape(ys, [-1, 1])
derived_size = tf.shape(ys)[0]
indices = tf.reshape(tf.range(0, derived_size, 1), [-1, 1])
sparse_labels = tf.cast(sparse_labels, tf.int32)
concated = tf.concat(1, [indices, sparse_labels])
outshape = tf.concat(0, [tf.reshape(derived_size, [1]), tf.reshape(num_labels, [1])])
labels = tf.sparse_to_dense(concated, outshape, 1.0, 0.0)
print(labels.get_shape())
labels = np.asarray(labels)

In [ ]:
ys=np.asarray(ys)
yzM=[];
import math
for r in range(4300):
    ys[r]= math.ceil(ys[r])
for r in range(ys.shape[0]):
    i=ys[r];
    yz=tf.zeros([1,10], tf.int32)
    yz[i]=1;
print(yz)

In [ ]:
error = np.linspace(0.0, 128.0**2, 100)
loss = error**2.0
plt.plot(error, loss)
plt.xlabel('error')
plt.ylabel('loss')

In [ ]:
error = np.linspace(0.0, 1.0, 100)
plt.plot(error, error**2, label='l_2 loss')
plt.plot(error, np.abs(error), label='l_1 loss')
plt.xlabel('error')
plt.ylabel('loss')
plt.legend(loc='lower right')

In [ ]:
# first compute the error, the inner part of the summation.
# This should be the l1-norm or l2-norm of the distance
# between each color channel.
error = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y_pred, labels=Y))
optimizer = tf.train.AdamOptimizer().minimize(error)

In [ ]:
correct_prediction = tf.equal(tf.argmax(Y_pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))


In [ ]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())

# %% We'll train in minibatches and report accuracy:
batch_size = 100
n_epochs = 100
with tf.Session() as sess:
    # Here we tell tensorflow that we want to initialize all
    # the variables in the graph so we can use them
    sess.run(tf.initialize_all_variables())

    # Fit all training data
    

    prev_training_cost = 0.0
    for epoch_i in range(n_epochs):
        for (x, y) in zip(x_train, y_train):
            sess.run(optimizer, feed_dict={X: xs, Y: labels})

        training_cost = sess.run(
            cost, feed_dict={X: x_train, Y: y_train})
        print(training_cost)

In [ ]:
# Finally, compute the cost, as the mean error of the batch.
# This should be a single value.
cost = ...
assert(cost.get_shape().as_list() == [])

In [ ]:
# Initialize all your variables and run the operation with your session
sess.run(tf.initialize_all_variables())

# Optimize over a few iterations, each time following the gradient
# a little at a time
imgs = []
costs = []
gif_step = n_iterations // 10
step_i = 0

for it_i in range(n_iterations):
    
    # Get a random sampling of the dataset
    idxs = np.random.permutation(range(len(xs)))
    
    # The number of batches we have to iterate over
    n_batches = len(idxs) // batch_size
    
    # Now iterate over our stochastic minibatches:
    for batch_i in range(n_batches):
         
        # Get just minibatch amount of data
        idxs_i = idxs[batch_i * batch_size: (batch_i + 1) * batch_size]

        # And optimize, also returning the cost so we can monitor
        # how our optimization is doing.
        training_cost = sess.run(
            [cost, optimizer],
            feed_dict={X: xs[idxs_i], Y: ys[idxs_i]})[0]

    # Also, every 20 iterations, we'll draw the prediction of our
    # input xs, which should try to recreate our image!
    if (it_i + 1) % gif_step == 0:
        costs.append(training_cost / n_batches)
        ys_pred = Y_pred.eval(feed_dict={X: xs}, session=sess)
        img = np.clip(ys_pred.reshape(img.shape), 0, 1)
        imgs.append(img)
        # Plot the cost over time
        fig, ax = plt.subplots(1, 2)
        ax[0].plot(costs)
        ax[0].set_xlabel('Iteration')
        ax[0].set_ylabel('Cost')
        ax[1].imshow(img)
        fig.suptitle('Iteration {}'.format(it_i))
        plt.show()